In [ ]:
import requests
import json
import pandas as pd
import boto3
from io import StringIO

## 일별 품목별 소매가격 정보 데이터 수집
* 데이터 출처: (농산물유통정보-KAMIS) https://www.kamis.or.kr/customer/reference/openapi_list.do?action=detail&boardno=2
* 선별한 주요 농산물 품목의 일별 소매가격 정보 데이터만 수집

In [ ]:
key = "22589f09-134f-44ec-adf6-ac3d07b2a75a" # api key
id = "4953" # user id
url = "http://www.kamis.or.kr/service/price/xml.do?action=periodProductList"

파라미터 및 데이터프레임 설정

In [ ]:
columns = ["itemcategorycode", "itemcode", "countyname", "date", "price"]
df_daily_item_prices = pd.DataFrame(columns=columns)

In [ ]:
params = {
    "p_convert_kg_yn": "N",
    "p_cert_key": key,
    "p_cert_id": id,
    "p_returntype": "json",
    "p_productclscode": "01",
    "p_startday": "2024-10-25",
    "p_endday": "2024-11-01",
}
itemcategorycode_list = [100, 400, 200, 100, 200, 200, 100, 200, 200, 200, 200]
itemcode_list = [151, 411, 225, 111, 232, 242, 152, 246, 245, 258, 221]
item_cnt = len(itemcode_list)

API 호출 및 데이터 전처리

In [ ]:
for i in range(item_cnt): #품목마다 접근
    item_params = {
        "p_itemcategorycode": str(itemcategorycode_list[i]),
        "p_itemcode": str(itemcode_list[i])
    }
    params.update(item_params)
    
    response = requests.get(url, params=params)
    data = response.json() # 특정 품목의 날짜별 데이터
    #print(data['condition'])
    for item in data['data']['item']: #날짜별 평균 데이터에 접근
        if item['countyname'] != "평균":
            continue
        record = []
        record.append(data['condition'][0]['p_itemcategorycode'])
        record.append(data['condition'][0]['p_itemcode'])
        record.append("전국")
        
        month, day = item['regday'].split('/')
        record_date = f"2024-{month}-{day}"
        record.append(record_date)
        
        record.append(int(item["price"].replace(",", "")))
        #print(record)
        df_daily_item_prices.loc[len(df_daily_item_prices)] = record #df에 삽입

## 월별 품목별 소매가격 정보 데이터 수집
* 데이터 출처: (농산물유통정보-KAMIS) https://www.kamis.or.kr/customer/reference/openapi_list.do?action=detail&boardno=3
* 선별한 주요 농산물 품목의 월별 소매가격 정보 데이터만 수집

In [3]:
# test_requset
url = 'http://www.kamis.or.kr/service/price/xml.do?action=monthlySalesList&p_yyyy=2024&p_period=5&p_itemcategorycode=100&p_itemcode=151&p_kindcode=00&p_graderank=2&p_countycode=1101&p_convert_kg_yn=N&p_cert_key=b4dc538e-2321-4b9d-aff7-f5c410926672&p_cert_id=4954&p_returntype=json'
response = requests.get(url)
data = response.json()
data

{'condition': [{'p_cert_id': '4954',
   'p_cert_key': 'b4dc538e-2321-4b9d-aff7-f5c410926672',
   'p_returntype': 'json',
   'p_yyyy': '2024',
   'p_period': '5',
   'p_itemcategorycode': '100',
   'p_itemcode': '151',
   'p_kindcode': '00',
   'p_graderank': '2',
   'p_countycode': '1101'}],
 'error_code': '000',
 'price': [{'productclscode': '02',
   'caption': '중도매인 판매가격 > 식량작물 > 고구마 > 밤 > 중품 > 10kg',
   'item': [{'yyyy': '2024',
     'm1': '26,600',
     'm2': '28,668',
     'm3': '30,440',
     'm4': '30,695',
     'm5': '34,865',
     'm6': '32,484',
     'm7': '22,722',
     'm8': '25,329',
     'm9': '25,661',
     'm10': '23,265',
     'm11': '24,300',
     'm12': '-',
     'yearavg': '27,942'},
    {'yyyy': '2023',
     'm1': '24,540',
     'm2': '24,600',
     'm3': '21,673',
     'm4': '21,895',
     'm5': '25,280',
     'm6': '30,157',
     'm7': '33,095',
     'm8': '34,091',
     'm9': '25,579',
     'm10': '22,000',
     'm11': '22,000',
     'm12': '22,568',
     'yeara

In [ ]:
# 필수 request parameters test
url = 'http://www.kamis.or.kr/service/price/xml.do?action=monthlySalesList&p_yyyy=2024&p_period=5&p_itemcategorycode=100&p_itemcode=151&p_graderank=2&p_convert_kg_yn=N&p_cert_key=b4dc538e-2321-4b9d-aff7-f5c410926672&p_cert_id=4954&p_returntype=json'
response = requests.get(url)
data = response.json()
data

{'condition': [{'p_cert_id': '4954',
   'p_cert_key': 'b4dc538e-2321-4b9d-aff7-f5c410926672',
   'p_returntype': 'json',
   'p_yyyy': '2024',
   'p_period': '5',
   'p_itemcategorycode': '100',
   'p_itemcode': '151',
   'p_kindcode': [],
   'p_graderank': '2',
   'p_countycode': []}],
 'error_code': '000',
 'price': [{'productclscode': '02',
   'caption': '중도매인 판매가격 > 식량작물 > 고구마 > 전체 > 중품 > 10kg',
   'item': [{'yyyy': '2024',
     'm1': '28,240',
     'm2': '29,178',
     'm3': '31,213',
     'm4': '31,735',
     'm5': '35,104',
     'm6': '30,494',
     'm7': '23,479',
     'm8': '24,579',
     'm9': '25,609',
     'm10': '24,425',
     'm11': '24,460',
     'm12': '-',
     'yearavg': '28,323'},
    {'yyyy': '2023',
     'm1': '25,158',
     'm2': '25,312',
     'm3': '23,565',
     'm4': '24,978',
     'm5': '26,997',
     'm6': '30,510',
     'm7': '31,490',
     'm8': '35,819',
     'm9': '29,843',
     'm10': '25,452',
     'm11': '25,785',
     'm12': '26,104',
     'yearavg': 

### 정보 요청을 위한 파라미터

In [1]:
base_url = 'http://www.kamis.or.kr/service/price/xml.do?action=monthlySalesList'

# 고정 파라미터
params = {
    "p_yyyy": "2024", # 2024년도까지
    "p_period": "5", # 2019년도부터
    "p_convert_kg_yn": "N", # kg 단위 환산 여부
    "p_cert_key": "b4dc538e-2321-4b9d-aff7-f5c410926672",  # 인증키
    "p_cert_id": "4954", # 요청자 ID
    "p_returntype": "json" # 반환 데이터 형식
}

item_params = [
    {"p_itemcategorycode": "100", "p_itemcode": "111"}, # 쌀 20kg
    {"p_itemcategorycode": "100", "p_itemcode": "151"}, # 고구마
    {"p_itemcategorycode": "100", "p_itemcode": "152"}, # 감자
    {"p_itemcategorycode": "200", "p_itemcode": "221"}, # 수박
    {"p_itemcategorycode": "200", "p_itemcode": "225"}, # 토마토
    {"p_itemcategorycode": "200", "p_itemcode": "232"}, # 당근
    {"p_itemcategorycode": "200", "p_itemcode": "242"}, # 풋고추
    {"p_itemcategorycode": "200", "p_itemcode": "245"}, # 양파
    {"p_itemcategorycode": "200", "p_itemcode": "246"}, # 파
    {"p_itemcategorycode": "200", "p_itemcode": "258"}, # 깐마늘(국산)
    {"p_itemcategorycode": "400", "p_itemcode": "411"} # 사과
]

In [ ]:

def fetch_monthly_data(base_url, params, item_params):
    all_data =[]

    for item in item_params:
        # 개별 품목 파라미터를 기존 params에 업데이트
        params.update(item)
        response = requests.get(base_url, params=params)
        data = response.json()
        #print(data)
        print(response.status_code)


        condition = data['condition'][0] # request parameter 정보 포함
        
        # 품목 그룹코드
        item_category_code = condition['p_itemcategorycode']
        # 품목 코드
        item_code = condition['p_itemcode']

        if data['price']:
            # 소매 데이터
            retail_sale = [entry for entry in data['price'] if entry['productclscode'] == '01']
        
            if 'item' in retail_sale[0]:
                # 데이터로 저장할 가격데이터
                prices = retail_sale[0]['item']

                for price in prices:
                    record = {
                        'itemcategorycode': item_category_code,
                        'itemcode': item_code,
                        'year': price['yyyy'],
                        'price_yearavg': price.get('yearavg', '-')
                    }
                    
                    for i in range(1, 13):
                        month_key = f'm{i}'
                        record[f'price_m{i}'] = price.get(month_key, '-') # 값이 존재하지 않을 경우 '-'
                    
                    all_data.append(record)
            else:
                    print(f"No retail sale data found for item: {item}")
        
        else:
            print(f"No price data found for item: {item}")


    df = pd.DataFrame(all_data)

    return df

    

In [ ]:
# 데이터 수집
df_monthly_item_prices = fetch_monthly_data(base_url=base_url, params=params, item_params=item_params)
df_monthly_item_prices

200
200
200
200
200
200
200
200
200
200
200


,itemcategorycode,itemcode,year,price_yearavg,price_m1,price_m2,price_m3,price_m4,price_m5,price_m6,price_m7,price_m8,price_m9,price_m10,price_m11,price_m12
0,100,111,2024,"40,616","43,281","42,450","40,580","39,542","40,982","41,105","40,609","39,424","38,609","39,279","41,208",-
1,100,111,2023,"46,333","50,576","50,807","51,436","51,479","51,779","42,132","41,335","42,152","42,424","45,538","43,205","43,640"
2,100,111,2022,"51,336","53,450","52,597","52,459","52,037","51,221","50,990","50,643","49,228","48,397","49,806","53,284","51,900"
3,100,111,2021,"59,080","59,979","60,374","59,990","59,991","61,104","61,247","61,725","61,327","57,327","55,322","55,381","55,135"
4,100,111,2020,"53,638","51,721","51,724","51,551","51,498","51,616","51,708","51,757","52,366","52,749","57,841","58,906","60,453"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,400,411,2023,"26,217","23,693","22,917","22,847","23,009","24,255","24,532","28,068","26,580","30,516","33,514","27,287","28,026"
62,400,411,2022,"26,934","26,600","26,646","27,331","27,922","28,386","28,803","29,895","30,647","27,011","25,383","22,566","21,935"
63,400,411,2021,"30,583","30,411","34,595","33,237","34,302","33,778","33,485","33,331","28,337","26,269","24,245","25,791","26,075"
64,400,411,2020,"24,249","19,173","20,177","19,818","20,403","21,497","22,988","26,010","25,567","30,874","29,844","27,743","27,242"


### 데이터 전처리
* 가격 데이터는 integer로
* '-'으로 표시된 결측치는 None으로 변경

In [ ]:
# 가격 데이터 정수 타입으로 변경
def preprocessing_monthly(df):

    months = [f'price_m{i}' for i in range(1, 13)]

    for month in months:
        df[month] = df[month].replace('-', None) # -은 None으로 변경
        df[month] = df[month].replace(',', '') # 쉼표 제거 후 정수형으로 변환
        df[month] = df[month].astype('Int64')

    df['price_yearavg'] = df["price_yearavg"].replace('-', None)
    df['price_yearavg'] = df["price_yearavg"].replace(',', '')
    df['price_yearavg'] = df["price_yearavg"].astype('Int64')

    return df



In [ ]:
df_monthly_item_prices = preprocessing_monthly(df_monthly_item_prices)
df_monthly_item_prices

,itemcategorycode,itemcode,year,price_yearavg,price_m1,price_m2,price_m3,price_m4,price_m5,price_m6,price_m7,price_m8,price_m9,price_m10,price_m11,price_m12
0,100,111,2024,40616,43281,42450,40580,39542,40982,41105,40609,39424,38609,39279,41208,<NA>
1,100,111,2023,46333,50576,50807,51436,51479,51779,42132,41335,42152,42424,45538,43205,43640
2,100,111,2022,51336,53450,52597,52459,52037,51221,50990,50643,49228,48397,49806,53284,51900
3,100,111,2021,59080,59979,60374,59990,59991,61104,61247,61725,61327,57327,55322,55381,55135
4,100,111,2020,53638,51721,51724,51551,51498,51616,51708,51757,52366,52749,57841,58906,60453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,400,411,2023,26217,23693,22917,22847,23009,24255,24532,28068,26580,30516,33514,27287,28026
62,400,411,2022,26934,26600,26646,27331,27922,28386,28803,29895,30647,27011,25383,22566,21935
63,400,411,2021,30583,30411,34595,33237,34302,33778,33485,33331,28337,26269,24245,25791,26075
64,400,411,2020,24249,19173,20177,19818,20403,21497,22988,26010,25567,30874,29844,27743,27242


# 지역별 품목별 소매 가격 정보
* 데이터 출처: (농산물유통정보-KAMIS) https://www.kamis.or.kr/customer/reference/openapi_list.do?action=detail&boardno=14
* 선별한 주요 농수산물 품목의 지역별 소매가격을 조회일자, 1주일 전, 1달 전, 1년 전의 가격 정보 데이터를 수집
* 지역은 서울 + 광역시 + 제주로 한정
* 수박 품목은 3개의 도시에서 가격 정보가 없으므로 제외 했음.

### 정보 요청을 위한 파라미터

In [ ]:

base_url = 'http://www.kamis.or.kr/service/price/xml.do?action=ItemInfo'

# 고정 request parameter 
params = {
    "p_productclscode": "01", # 소매 (02: 도매)
    "p_regday": "2024-11-01", # 조회 일자
    "p_cert_key": "b4dc538e-2321-4b9d-aff7-f5c410926672",  # 인증키
    "p_cert_id": "4954", # 요청 id
    "p_returntype": "json" # 반환 타입
}

# 품목 request parameter
item_params = [
    {"p_itemcategorycode": "100", "p_itemcode": "111"}, # 쌀 20kg
    {"p_itemcategorycode": "100", "p_itemcode": "151"}, # 고구마
    {"p_itemcategorycode": "100", "p_itemcode": "152"}, # 감자
    {"p_itemcategorycode": "200", "p_itemcode": "225"}, # 토마토
    {"p_itemcategorycode": "200", "p_itemcode": "232"}, # 당근
    {"p_itemcategorycode": "200", "p_itemcode": "242"}, # 풋고추
    {"p_itemcategorycode": "200", "p_itemcode": "245"}, # 양파
    {"p_itemcategorycode": "200", "p_itemcode": "246"}, # 파
    {"p_itemcategorycode": "200", "p_itemcode": "258"}, # 깐마늘(국산)
    {"p_itemcategorycode": "400", "p_itemcode": "411"} # 사과
]

# 지역 코드 request parameter
county_codes = {
    "서울": "1101", "부산": "2100", "대구": "2200", "인천": "2300", "광주": "2401",
    "대전": "2501", "울산": "2601", "제주": "3911"
}

In [ ]:
def fetch_county_data(base_url, params, item_params, county_codes):
    # 데이터 수집을 위한 빈 리스트
    data_list = []

    # 각 품목 및 지역에 대해 데이터 요청
    for item in item_params:
        for county_name, county_code in county_codes.items():
            # 요청 URL 생성
            url = f"{base_url}&p_productclscode=01&p_countycode={county_code}&p_regday={params['p_regday']}&p_itemcategorycode={item['p_itemcategorycode']}&p_itemcode={item['p_itemcode']}&p_cert_key={params['p_cert_key']}&p_cert_id={params['p_cert_id']}&p_returntype=json"
            # print(url)
            # 데이터 요청
            response = requests.get(url)
            response_data = response.json()
            #print(county_name, county_code)
            
            condition = response_data['condition'][0]
            item_category_code = condition['p_itemcategorycode']
            item_code = condition['p_itemcode']
            reg_day = condition['p_regday']

    # response_data['data']['item']의 countyname이 request parameter에 사용한 countyname과 동일한 데이터 가져오기 
            region_data = [region for region in response_data['data']['item'] if region['countyname'] == county_name][0]
            
            data_list.append({
                'itemcategorycode': item_category_code,
                'itemcode': item_code,
                'date': reg_day,
                'countyname': region_data['countyname'],
                'price': region_data['price'],
                'weekprice': region_data['weekprice'],
                'monthprice': region_data['monthprice'],
                'yearprice': region_data['yearprice']
            })

    # DataFrame으로 변환
    df = pd.DataFrame(data_list)

    return df

In [ ]:
df_county_item_prices = fetch_county_data(base_url=base_url, params=params, item_params=item_params, county_codes=county_codes)
df_county_item_prices

### 데이터 전처리
* 가격 데이터는 integer로
* '-'으로 표시된 결측치는 None으로 변경

In [ ]:
def preprocessing_county(df):
    for column in ['price', 'weekprice', 'monthprice', 'yearprice']:
        df[column] = df[column].replace('-', None)
        df[column] = df[column].str.replace(',', '')
        df[column] = df[column].astype('Int64')

    return df


In [ ]:
df_county_item_prices = preprocessing_county(df_county_item_prices)
df_county_item_prices

## S3에 Extract, Transform 완료된 테이블들을 로드

In [ ]:
df_list = ['df_daily_item_prices',
            'df_monthly_item_prices',
            'df_county_item_prices']

In [ ]:


#AWS 자격증명 설정
aws_access_key_id = '-' # aws_access_key_입력
aws_secret_access_key = '-' # aws_secret_access_key_입력
region_name = '-' # region_name_입력

#S3 클라이언트 생성
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)




In [ ]:
for df_string in df_list:
    # 데이터프레임 객체 가져오기
    df = globals()[df_string]

    # DataFrame을 CSV 형식으로 변환
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')
    bucket_name = '4ward-food-bucket'
    file_name = df_string[3:] # 파일명 'df_' 제거
    file_path = 'food_data/{}.csv'.format(file_name)
    
    s3.put_object(Bucket=bucket_name, Key=file_path, Body=csv_buffer.getvalue())
    print(df_string, "업로드 완료")
print('끝')

